In [1]:
# Data を読み込んで把握する
import pandas as pd

# Jim の利用履歴(2018/04 ~ 2019/03)の Data
uselog = pd.read_csv('sample_code/chapter_3/use_log.csv')
print(len(uselog))
uselog.head()

197428


,log_id,customer_id,usedate
0,L00000049012330,AS009373,2018-04-01
1,L00000049012331,AS015315,2018-04-01
2,L00000049012332,AS040841,2018-04-01
3,L00000049012333,AS046594,2018-04-01
4,L00000049012334,AS073285,2018-04-01


### uselog(利用履歴)
- 顧客ID、利用日を含んだ３列のみの Simple な Data.
- どの顧客がいつ Jim を利用したのかがわかる。
- 件数は、197,428件と縦に長い。

In [2]:
# 2019/03 末時点での会員 Data
customer = pd.read_csv('sample_code/chapter_3/customer_master.csv')
print(len(customer))
customer.head()

4192


,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted
0,OA832399,XXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0
1,PL270116,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0
2,OA974876,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0
3,HD024127,XXXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0
4,HD661448,XXXXX,C03,F,2015-05-01 00:00:00,NaN,CA1,0


### customer(会員)
- 顧客ID、名前、会員 Class、性別、登録日等の情報が含まれている。
- 名前は Masking されており、名前だけでは個人は特定できないようになっている。
- is_delete 列は、2019年3月時点で大会している User を System 的に素早く検索するための列
   => **uselog と customer_id の紐づけが可能。**
- すでい退会済みの User も含めて4,192人。
- class, campaign_id 列を用いれば、self data と結合できる。

In [4]:
# 会員区分 Data
class_master = pd.read_csv('sample_code/chapter_3/class_master.csv')
print(len(class_master))
class_master.head()

3


,class,class_name,price
0,C01,オールタイム,10500
1,C02,デイタイム,7500
2,C03,ナイト,6000


In [5]:
# Campaign 区分 Data
campaign_master = pd.read_csv('sample_code/chapter_3/campaign_master.csv')
print(len(campaign_master))
campaign_master.head()

3


,campaign_id,campaign_name
0,CA1,通常
1,CA2,入会費半額
2,CA3,入会費無料


### 加工にあたり、主にする Data
- customer(顧客 Data)
- uselog(利用履歴)

が考えらえる。

まず、Data 数の少ない customer(顧客 Data)を主に考えていく。

一旦、uselog(利用履歴) data を置いておいて customer(顧客 Data)を整形し、どのような顧客が何人くらいいるのか等の全体像を掴む。

In [6]:
# 顧客 Data を整形する

customer_join = pd.merge(customer, class_master, on='class', how='left')
customer_join = pd.merge(customer_join, campaign_master, on='campaign_id', how='left')
customer_join

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name
0,OA832399,XXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0,オールタイム,10500,通常
1,PL270116,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0,オールタイム,10500,通常
2,OA974876,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0,オールタイム,10500,通常
3,HD024127,XXXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0,オールタイム,10500,通常
4,HD661448,XXXXX,C03,F,2015-05-01 00:00:00,NaN,CA1,0,ナイト,6000,通常
...,...,...,...,...,...,...,...,...,...,...,...
4187,HD676663,XXXX,C01,M,2019-03-14 00:00:00,NaN,CA1,0,オールタイム,10500,通常
4188,HD246549,XXXXX,C01,F,2019-03-14 00:00:00,NaN,CA1,0,オールタイム,10500,通常
4189,GD037007,XXXXX,C03,M,2019-03-14 00:00:00,NaN,CA1,0,ナイト,6000,通常
4190,OA953150,XXXXX,C01,M,2019-03-14 00:00:00,NaN,CA1,0,オールタイム,10500,通常


In [7]:
print(len(customer))
print(len(customer_join))

4192
4192


会員区分や金額等がわかるように Data 整形完了。
Data 件数も、Join 前後で変化がないことを確認。

欠損値の確認をする。

In [8]:
customer_join.isnull().sum()

customer_id         0
name                0
class               0
gender              0
start_date          0
end_date         2842
campaign_id         0
is_deleted          0
class_name          0
price               0
campaign_name       0
dtype: int64

- end_date 以外は欠損値が0
- end_date に欠損値が入っていること以外は比較的綺麗な Data.
  退会していない User は退会日である end_date を保持していないため、欠損値となっていると思われる。

